# Uisnig FinBERT for Sentiment Analysis

Here, we will use FinBERT for sentiment analysis of the news in `media.csv`. We choose FinBERT because it is a pre-trained model that is specifically trained on financial news. We compared it with other models and found that it performed the best in the notebook `SA(NLP-compare).ipynb`.

## Loading the data from exploratory_data_analysis

In [1]:
import pandas as pd

media_data = pd.read_csv("../exploratory_data_analysis/data/media.csv", index_col = 0, parse_dates = ['pub_date'])

FileNotFoundError: [Errno 2] No such file or directory: '../test_data/media.csv'

In [2]:
## fix a seed for reproducibility
from transformers import set_seed

set_seed(2023)

/Users/parhamhamidi/opt/anaconda3/envs/NLP/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Using ProsusAI/finbert

See https://huggingface.co/ProsusAI/finbert

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer_finbert = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model_finbert = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

/Users/parhamhamidi/opt/anaconda3/envs/NLP/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from transformers import pipeline

analyzer_finbert = pipeline("sentiment-analysis", model= model_finbert , tokenizer = tokenizer_finbert)

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [5]:
## make a function that reades the label of the analyzer_finbert result
## if the label is positive, it returns the analyzer_finbert score
## if the label is negative, it returns the negative of the analyzer_finbert score

def analyzer_finbert_score(text):
    result = analyzer_finbert(text)
    if result[0].get('label') == 'positive':
        return result[0].get('score')
    if result[0].get('label') == 'negative':
        return -result[0].get('score')
    else:
        return 0

In [7]:
## make a function that reades the result of analyzer_finbert_score
## if the score is greater than 0.9 , it returns 2
## if the score is between 0.5 and 0.9, it returns 1
## if the score is between -0.5 and 0.5, it returns 0
## if the score is between -0.9 and -0.5, it returns -1
## if the score is less than -0.9, it returns -2

def analyzer_finbert_bin(text):
    score = analyzer_finbert_score(text)
    if score > 0.9:
        return 2
    if score > 0.5:
        return 1
    if score > -0.5:
        return 0
    if score > -0.9:
        return -1
    else:
        return -2


In [9]:
## Apply analyzer_fin pipeline to the text of each article and recording the sentiment scores in a new column

# initialize a list to store the sentiment scores
sentiment_scores = []

# loop through each article
for text in media_data['text']:
    # apply the sentiment analysis pipeline to the abstract
    sentiment_scores.append(analyzer_finbert_score(text))
    
# add the sentiment scores to the media data
media_data['finbert-sentiment-score'] = sentiment_scores

In [10]:
## We can apply analyzer_finbert_bin pipeline to the text of each article and recording the sentiment scores in a new column

# initialize a list to store the sentiment scores
sentiment_scores = []

# loop through each article
for text in media_data['text']:
    # apply the sentiment analysis pipeline to the abstract
    sentiment_scores.append(analyzer_finbert_bin(text))
    
# add the sentiment scores to the media data
media_data['finbert-bin'] = sentiment_scores

In [11]:
media_data.sample(10)

,pub_date,abstract,lead_paragraph,snippet,headline.main,text,Polarity,Sentiment,NLP_fin-sentiment-text,finbert-sentiment-score,finbert-bin
56,2022-05-20 15:50:25+00:00,"Flush with cash, Facebook, Apple, Amazon, Micr...","SAN FRANCISCO — Apple, Amazon, Microsoft and t...","Flush with cash, Facebook, Apple, Amazon, Micr...",Big Tech Is Getting Clobbered on Wall Street. ...,"Flush with cash, Facebook, Apple, Amazon, Micr...",0.870781,0.830735,-0.974612,-0.915605,-2
12,2022-09-16 09:00:25+00:00,Need to find a restaurant or figure out how to...,When Ja’Kobi Moore decided to apply this year ...,Need to find a restaurant or figure out how to...,"For Gen Z, TikTok Is the New Search Engine",Need to find a restaurant or figure out how to...,-0.590656,0.774193,-0.999481,0.000000,0
68,2022-04-08 14:34:29+00:00,New research finds companies are starting to r...,"As a middle school student in New York, Shekin...",New research finds companies are starting to r...,A 4-Year Degree Isn’t Quite the Job Requiremen...,New research finds companies are starting to r...,0.214991,0.123951,-0.683587,0.000000,0
0,2022-10-25 20:37:03+00:00,Google’s parent company reported earnings that...,"Even Alphabet, the parent company of Google an...",Google’s parent company reported earnings that...,Alphabet’s Profit Drops 27 Percent From a Year...,Google’s parent company reported earnings that...,-0.478889,0.504897,-0.998828,-0.973848,-2
82,2022-01-20 21:10:22+00:00,"Mustafa Suleyman, who played a key role in the...","Mustafa Suleyman, a pioneer in the field of ar...","Mustafa Suleyman, who played a key role in the...",DeepMind co-founder leaves Google after a rock...,"Mustafa Suleyman, who played a key role in the...",0.812955,0.340731,-0.999767,0.000000,0
66,2022-04-12 18:59:03+00:00,The company’s first consumer protection lawsui...,"In a first for the tech giant, Google filed a ...",The company’s first consumer protection lawsui...,"In a First, Google Goes After Puppy Fraud in C...",The company’s first consumer protection lawsui...,-0.617800,0.890100,-0.999104,-0.626287,-1
91,2022-01-10 15:00:05+00:00,The latest tranche totals about 200. They are ...,Google wrongly claimed attorney-client privile...,The latest tranche totals about 200. They are ...,Google must turn over more documents in a labo...,The latest tranche totals about 200. They are ...,-0.762656,0.638658,-0.999734,-0.827862,-1
44,2022-06-16 09:00:33+00:00,A video producer claims he was fired after he ...,"OREGON HOUSE, Calif. — In a tiny town in the f...",A video producer claims he was fired after he ...,How a Religious Sect Landed Google in a Lawsuit,A video producer claims he was fired after he ...,0.776261,0.865660,-0.931622,-0.839244,-1
46,2022-06-13 02:13:37+00:00,The tech giant admitted no wrongdoing as it re...,Google has settled a class-action lawsuit that...,The tech giant admitted no wrongdoing as it re...,Google Agrees to Pay $118 Million to Settle Pa...,The tech giant admitted no wrongdoing as it re...,-0.059274,0.316655,0.998235,0.000000,0
20,2022-08-30 20:57:56+00:00,The social media app is not available on Googl...,Google said on Tuesday that it would not distr...,The social media app is not available on Googl...,Google Says Trump’s Truth Social Must Scrub Vi...,The social media app is not available on Googl...,0.409295,0.261195,-0.998796,-0.778782,-1


In [12]:
media_data.describe()

,Polarity,Sentiment,NLP_fin-sentiment-text,finbert-sentiment-score,finbert-bin
count,100.000000,100.000000,100.000000,100.000000,100.000000
mean,-0.013616,0.549705,-0.537430,-0.333282,-0.590000
std,0.595266,0.269949,0.790210,0.477305,0.853927
min,-0.998627,0.029718,-0.999880,-0.975459,-2.000000
25%,-0.481270,0.315349,-0.998546,-0.833797,-1.000000
50%,-0.021043,0.582934,-0.981465,0.000000,0.000000
75%,0.522078,0.769434,-0.656894,0.000000,0.000000
max,0.974263,0.994465,0.999672,0.886748,1.000000


## Saving the results to exploratory_data_analysis

In [13]:
## save the media data
media_data.to_csv("../exploratory_data_analysis/data/media.csv")

## Adding Sentiment Analysis the test and train data

In [2]:
## Loading test and train media data
import pandas as pd

media_test = pd.read_csv("../data/test/media.csv", index_col = 0, parse_dates = ['pub_date'])
media_train = pd.read_csv("../data/train/media.csv", index_col = 0, parse_dates = ['pub_date'])


Apply `analyzer_finbert_score` to the `text` of each article in media_test and media_train and recording the sentiment scores in new column `finbert-sentiment-score`.

In [8]:


## creating media_train['finbert-sentiment-score'] column
# initialize a list to store the sentiment scores
sentiment_scores_train = []

# loop through each article
for text in media_train['text']:
    # apply the sentiment analysis pipeline to the text
    sentiment_scores_train.append(analyzer_finbert_score(text))
    
# add the sentiment scores to the media data
media_train['finbert-sentiment-score'] = sentiment_scores_train



In [ ]:


## creating media_test['finbert-sentiment-score'] column
# initialize a list to store the sentiment scores
sentiment_scores_test = []

# loop through each article
for text in media_test['text']:
    # apply the sentiment analysis pipeline to the text
    sentiment_scores_test.append(analyzer_finbert_score(text))
    
# add the sentiment scores to the media data
media_test['finbert-sentiment-score'] = sentiment_scores_test



Apply `analyzer_finbert_bin` to the `text` of each article in media_test and media_train and recording the sentiment scores in new column called `finbert-bin`

In [9]:

# initialize a list to store the sentiments
sentiment_bin_train = []

# loop through each article
for text in media_train['text']:
    # apply the sentiment analysis pipeline to the text
    sentiment_bin_train.append(analyzer_finbert_bin(text))
    
# add the sentiment scores to the media data
media_train['finbert-bin'] = sentiment_bin_train

In [10]:
# initialize a list to store the sentiments
sentiment_bin_test = []

# loop through each article
for text in media_test['text']:
    # apply the sentiment analysis pipeline to the text
    sentiment_bin_test.append(analyzer_finbert_bin(text))
    
# add the sentiment scores to the media data
media_test['finbert-bin'] = sentiment_bin_test

In [11]:
media_train.head()

,pub_date,word_count,type_of_material,_id,abstract,lead_paragraph,snippet,headline.main,text,relevance_rank,keyword_counts,polarity,subjectivity,finbert-sentiment-score,finbert-bin
0,2020-10-15 20:10:10+00:00,438,News,nyt://article/02669a7b-5dfa-5c38-92bc-10ebcc39...,Facebook and Twitter clamped down on an unsubs...,In all the uproar over how tech companies have...,Facebook and Twitter clamped down on an unsubs...,"In Hubbub Over New York Post Report, YouTube S...",Facebook and Twitter clamped down on an unsubs...,4,0,0.032983,0.455107,-0.943060,-2
1,2021-08-31 17:07:21+00:00,191,News,nyt://article/026a2f96-48f3-5896-916e-badc358d...,It is the latest company to push back plans fo...,Google is pushing back its return-to-office da...,It is the latest company to push back plans fo...,Google delays its return to office until January.,It is the latest company to push back plans fo...,4,2,0.228000,0.482000,-0.727857,-1
2,2021-04-12 16:50:18+00:00,607,Letter,nyt://article/02b7abdf-e925-5941-b577-9c2737ba...,Readers discuss formative experiences in the w...,To the Editor:,Readers discuss formative experiences in the w...,My Job and I: Is This a Love Letter?,Readers discuss formative experiences in the w...,1,2,0.500000,0.600000,0.000000,0
3,2020-10-21 11:56:39+00:00,2057,News,nyt://article/0322b0d5-4d0a-50bf-93f5-2fe37e9d...,What you need to know about the antitrust case...,"This Nov. 17 and 18, DealBook opens its doors ...",What you need to know about the antitrust case...,What Did Google Do?,What you need to know about the antitrust case...,2,1,0.306250,0.682292,0.000000,0
4,2020-06-04 22:04:38+00:00,1268,News,nyt://article/044a14cb-41a2-54a5-aef6-273f5b33...,"To placate European regulators, the company st...","OAKLAND, Calif. — For the last few months, som...","To placate European regulators, the company st...",Google’s European Search Menu Draws Interest o...,"To placate European regulators, the company st...",1,3,-0.023580,0.137027,0.000000,0


In [12]:
media_test.head()

,pub_date,word_count,type_of_material,_id,abstract,lead_paragraph,snippet,headline.main,text,relevance_rank,keyword_counts,polarity,subjectivity,finbert-sentiment-score,finbert-bin
8,2022-05-02 09:00:17+00:00,1336,News,nyt://article/06d7c3d9-ecd5-5515-a936-9b7035c2...,The researchers are considered a key to the co...,Less than two years after Google dismissed two...,The researchers are considered a key to the co...,Another Firing Among Google’s A.I. Brain Trust...,The researchers are considered a key to the co...,1,2,-0.145000,0.590000,-0.919329,-2
22,2022-05-23 07:00:10+00:00,1649,News,nyt://article/11b22572-38cc-5663-a8c6-5a82a3ef...,"They moved from Russia, Ukraine, Belarus and o...","SAN FRANCISCO — Over the past five years, Andr...","They moved from Russia, Ukraine, Belarus and o...",Russian-Speaking Technologists Rebuild Their L...,"They moved from Russia, Ukraine, Belarus and o...",5,0,-0.105000,0.355000,0.000000,0
23,2022-05-31 16:37:15+00:00,1334,News,nyt://article/11df4af8-e254-587d-a195-1fccbb95...,The addition of one of the most widely spoken ...,LIMA — When Irma Alvarez Ccoscco heard that th...,The addition of one of the most widely spoken ...,‘Allinllachu.’ Google Translate Adds Quechua t...,The addition of one of the most widely spoken ...,5,2,0.114286,0.365476,0.000000,0
28,2022-11-17 13:32:18+00:00,1799,News,nyt://article/15c8aabc-5866-5ef6-91bd-9cfe8220...,The crypto exchange’s collapse is stoking fear...,The breadth of the global fallout from FTX’s c...,The crypto exchange’s collapse is stoking fear...,Investor Losses From FTX’s Implosion Are Growing,The crypto exchange’s collapse is stoking fear...,3,0,-0.231111,0.317778,-0.949574,-2
32,2022-09-28 17:25:57+00:00,529,News,nyt://article/1785c3ba-a570-50cb-8a3c-077b6496...,From more photo-based results to neighborhood ...,Google’s search engine looks a little differen...,From more photo-based results to neighborhood ...,Google to Make Search and Maps More ‘Immersive’,From more photo-based results to neighborhood ...,1,2,0.355682,0.581818,0.000000,0


Save the results to `media_test.csv` and `media_train.csv`

In [13]:
media_train.to_csv("../data/train/media.csv")
media_test.to_csv("../data/test/media.csv")